In [40]:
## Import libraries

import os, glob, pandas as pd
import string, re
import numpy as np
from tqdm import tqdm
from collections import Counter
import numbers
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from string import ascii_lowercase
import os 
import glob
import re
import requests
import shutil
import pandas as pd
import time
from nltk import ngrams
from operator import itemgetter
import pickle
from flashtext import KeywordProcessor
import itertools

In [8]:
os.chdir("C://Users//Ruben//Documents//Artikelen//Joris")
with open('stopwords-nl.txt', encoding = 'utf-8') as f:
    stopz = f.read().splitlines()

os.chdir("C://Users//Ruben//Documents//GitHub//dhl-ads//resources")
with open('occupations_dictionary.pkl', 'rb') as handle:
    occ_dict = pickle.load(handle)
    
list_words = []
for k,v in occ_dict.items():
    k = k
    v = [x for x in v if x not in stopz and len(x) > 3]
    
    for x in v:
        list_words.append(x)
    list_words.append(k)
    
keyword_processor = KeywordProcessor()
for w in list_words:
    keyword_processor.add_keyword(w)
    
os.chdir("C://Users//Ruben//Downloads//metadata//leco_set_80")
df = pd.read_csv('1880-1890.csv', sep = '\t')

def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    txt = txt.split(' ')
    return txt

df['clean'] = [clean_and_split_str(i) for i in df.ocr]

# Import QUAL Indicator list
os.chdir("C://Users//Ruben//Documents//GitHub//dhl-ads//resources")
with open('qualitative_indicators.txt', encoding = 'utf-8') as f:
    qual_words = f.read().splitlines()
    
# Import QUAL Indicator list
os.chdir("C://Users//Ruben//Documents//GitHub//dhl-ads//resources")
with open('wage_indicators.txt', encoding = 'utf-8') as f:
    wage_words = f.read().splitlines()

In [31]:
def ExtractWindows2(df, list_words, qual_words):
    alphabet_id = [c for c in ascii_lowercase] + [c + c for c in ascii_lowercase] + [c + c +c for c in ascii_lowercase]

    opd = dict()
    ctr = 0 # counter for showing progress

    for c,i in enumerate(df.clean):
        ctr += 1
        
        extracted_occupations = keyword_processor.extract_keywords(" ".join(i))
        extracted_occupations = [o for o in extracted_occupations if o in i]
        
        if len(extracted_occupations) > 0:     # check if there is an occupation in the advertisement

            for count,o in enumerate(extracted_occupations):                       # loop over occupations in ad-occupation list and select windows around 'em
                ind = i.index(o)
                sl = i[ind-12:ind+40]
                sl = ' '.join(sl)
                if 'loon ' in sl or 'salaris' in sl:               # if the word 'loon' is in the window; append [occupation:window] to list (not dictionary because of duplicate occupations)
                    output_list = [o, sl, df['id'][c], df['date'][c], df['image_url'][c], i, ind]
                    opd.update({str(c) + alphabet_id[count]:output_list})

        if ctr % 5000 == 0:                    # print the progress
            print(str(round(ctr / len(df.clean) * 100)) + "%")
    
    
    ## Convert output dictionary to dataframe (for clarity)
    dfa = pd.DataFrame([opd.keys() ,[v[3] for k,v in opd.items()], [v[0] for k,v in opd.items()], [v[1] for k,v in opd.items()],[v[4] for k,v in opd.items()], [' '.join(v[5]) for k,v in opd.items()],[v[6] for k,v in opd.items()],]).T
    dfa.columns = ['id','date', 'oc', 'window','image_url', 'ocr', 'occ_index']
    return dfa

In [32]:
dfr = ExtractWindows2(df, list_words, qual_words)

5%
11%
16%
22%
27%
32%
38%
43%
48%
54%
59%
65%
70%
75%
81%
86%
91%
97%


In [37]:
os.chdir("C://Users//Ruben//Downloads//metadata//ads_all_1870s")
df1 = pd.read_csv('1870-1872.csv', sep = '\t')
df2 = pd.read_csv('1872-1879.csv', sep = '\t')
df = df1.append(df2)

df.to_csv('1870-1879.csv', sep = "\t")

In [42]:
l = [c for c in ascii_lowercase]

In [43]:
keywords = [''.join(i) for i in itertools.product(l, repeat = 4)]

In [45]:
len(keywords)

456976